# COVID-19 Infographic 

Getting data for [Strongly-related COVID articles][https://meta.wikimedia.org/wiki/User:Diego_(WMF)/COVID-19_Articles_(All_but_Q5)]: 

- Total number of strongly-related COVID articles globally, both by language and combined languages
- Total number of languages that strongly-related COVID articles exist in 
- Total number of pageviews from Dec. 2019 - present on strongly-related COVID articles globally
- Total number of edits to strongly-related COVID articles globally
- Frequency of edits on strongly-related COVID articles globally 
- Total number of editors on all strongly-related COVID articles


In [1]:
import requests
import pandas as pd
import json
import matplotlib.pyplot as plt
import gzip
from wmfdata import hive,spark
import numpy as np

In [2]:
## load COVID related page list into hive table
filepath = "COVID-article-list-03_30_2020.csv"

In [3]:
hive.load_csv(
    filepath,
    field_spec="Wikidata string, project string, page string, url string, wikilink string",
    db_name="cchen",
    table_name="covid_pages",
)

### Language & Articles

In [3]:
## join canonical wiki table to get language and language code
related_pages = spark.run("""
    SELECT
      w.database_code, 
      p.project,
      w.language_code, 
      w.language_name,
      p.page, 
      p.url
    FROM cchen.covid_pages p 
      LEFT JOIN canonical_data.wikis w ON  CONCAT(p.project,'.org') =   w.domain_name """)

In [4]:
related_pages[0:5]

,database_code,project,language_code,language_name,page,url
0,afwiki,af.wikipedia,af,Afrikaans,Ernstige akute respiratoriese sindroom,https://af.wikipedia.org/wiki/Ernstige akute r...
1,arwiki,ar.wikipedia,ar,Arabic,متلازمة تنفسية حادة وخيمة,https://ar.wikipedia.org/wiki/متلازمة تنفسية ح...
2,astwiki,ast.wikipedia,ast,Asturian,Síndrome respiratoriu agudu grave,https://ast.wikipedia.org/wiki/Síndrome respir...
3,azbwiki,azb.wikipedia,azb,South Azerbaijani,سارس,https://azb.wikipedia.org/wiki/سارس
4,bgwiki,bg.wikipedia,bg,Bulgarian,Тежък остър респираторен синдром,https://bg.wikipedia.org/wiki/Тежък остър респ...


In [27]:
print('Total number of related COVID articles globally is %s' %len(related_pages))
print('Total number of languages that strongly-related COVID articles exist is %s' %related_pages["language_code"].nunique(dropna = True))

Total number of related COVID articles globally is 3372
Total number of languages that strongly-related COVID articles exist is 143


In [21]:
## count of articles by languages
related_page_lan = related_pages.groupby(['language_code', 'language_name']).size().reset_index(name='article_counts')
related_page_lan[0:5]

,language_code,language_name,counts
0,af,Afrikaans,4
1,als,Alsatian,2
2,am,Amharic,2
3,an,Aragonese,4
4,ang,Old English,1


In [30]:
related_page_lan.to_csv("articles_per_language.csv")

## Edits and Edit frequency

In [42]:
related_dict = related_pages.groupby('project')['page'].apply(list).to_dict()

In [44]:
import mwapi
import pandas as pd

def countRevisionsPerDay(page_name,project,startdate):
    """
    page_name: str, article title, ex: 'COVID-19'
    project: str, project id, ex: 'es.wikipedia'
    startdate: timestamp, counting from given day example '2020-01-01T00:00:00Z'
    
    """
    counterPerDay = {}
    
    session = mwapi.Session("https://%s.org" % project, user_agent="cchen@wikimedia.org - COVID-19 research")
    for response_doc in session.get(action='query', prop='revisions', titles=page_name, 
                                    rvprop=['ids', 'timestamp'], rvlimit=100, rvdir="newer", 
                                    formatversion=2, rvstart=startdate, continuation=True):
        for rev_doc in response_doc['query']['pages'][0]['revisions']:
            rev_id = rev_doc['revid']
            day = pd.to_datetime(rev_doc['timestamp']).strftime("%Y-%m-%d")
            counterPerDay[day] = counterPerDay.get(day,0)
            counterPerDay[day]  += 1
    output = pd.DataFrame.from_dict(counterPerDay,orient='index',columns=[page_name])
    output.index = pd.to_datetime(output.index)
    return output

In [43]:
#there are some errors/warnings. Don't worry, we are getting a lower-bound
perDayResults = {}
startDate = '2020-01-01T00:00:00Z'
for project, pages in related_dict.items():
    print(project)
    for page in pages:
        try:
            perDay = countRevisionsPerDay(page,project,startDate)
            perDayResults[project] = perDayResults.get(project,[])
            perDayResults[project].append(perDay)
        except:
            #print('error in %s %s' % (page,project))
            pass


be_x_old.wikipedia
ennews.wikipedia
atj.wikipedia
tl.wikipedia
hak.wikipedia
ast.wikipedia
itvoyage.wikipedia
wuu.wikipedia
enquote.wikipedia
itquote.wikipedia
am.wikipedia
hy.wikipedia
el.wikipedia
mwl.wikipedia
oc.wikipedia
sh.wikipedia
esversity.wikipedia
pap.wikipedia
ko.wikipedia
zh.wikipedia
frquote.wikipedia
sat.wikipedia
frvoyage.wikipedia
ukquote.wikipedia
mn.wikipedia
or.wikipedia
frnews.wikipedia
ptnews.wikipedia
ceb.wikipedia
se.wikipedia
sd.wikipedia
konews.wikipedia
bcl.wikipedia
zh_classical.wikipedia
fi.wikipedia
qu.wikipedia
esvoyage.wikipedia
mg.wikipedia
gl.wikipedia
tr.wikipedia
war.wikipedia
ro.wikipedia
id.wikipedia
esnews.wikipedia
ta.wikipedia
rue.wikipedia
eu.wikipedia
mnw.wikipedia
finews.wikipedia
pa.wikipedia
azb.wikipedia
uknews.wikipedia
ne.wikipedia
data.wikipedia
wa.wikipedia
lij.wikipedia
et.wikipedia
zh_yue.wikipedia
ug.wikipedia
zhnews.wikipedia
sw.wikipedia
frp.wikipedia
ha.wikipedia
eml.wikipedia
km.wikipedia
enversity.wikipedia
species.wikipedia
a

In [46]:
resultsPerProjectperDay = {}
for project, edits in perDayResults.items():
    resultsPerProjectperDay[project] = pd.concat(edits,axis=1)
    resultsPerProjectperDay[project].index = pd.to_datetime(resultsPerProjectperDay[project].index)

In [59]:
allEditsPerDay = []
for project, edits in resultsPerProjectperDay.items():
    allEditsPerDay.append(pd.DataFrame(edits.sum(axis=1)))
allEditsPerDay = pd.concat(allEditsPerDay,axis=1).sum(axis=1)

edits_df = pd.DataFrame({'date':allEditsPerDay.index, 'edit_count':allEditsPerDay.values})

In [64]:
## monthly edits
edits_df.groupby(pd.Grouper(key='date',freq='M')).sum()  


,edit_count
date,
2019-12-31,36.0
2020-01-31,29085.0
2020-02-29,62225.0
2020-03-31,248791.0


In [68]:
print('Total number of edits %s' % edits_df.edit_count.sum())
avgPerDay = round(edits_df.edit_count.sum()/(31+29+30))
print('Avg Edits per Day %s' % avgPerDay )
avgPerHour = round(edits_df.edit_count.sum()/(24*(31+29+30)))
print('Avg Edits per hour %s' % avgPerHour )

Total number of edits 340137.0
Avg Edits per Day 3779.0
Avg Edits per hour 157.0


## Editors

In [69]:
def countEditors(page_name,project,date):
    """
    page_name: str, article title, ex: 'COVID-19'
    project: str, project id, ex: 'es.wikipedia'
    date: timestamp, counting from given day example '2020-01-01T00:00:00Z'
    
    """
    editors = {} #cross project user name should be the same.
    ananoymous = {}
    counter = 0
    session = mwapi.Session("https://%s.org" % project, user_agent="dsaez@wikimedia.org - COVID-19 research")
    for response_doc in session.get(action='query', prop='revisions', titles=page_name, 
                                    rvprop=['ids', 'timestamp','user','userid'], rvlimit=100, rvdir="newer", 
                                    formatversion=2, rvstart=date, continuation=True):
        for rev_doc in response_doc['query']['pages'][0]['revisions']:
            rev_id = rev_doc['revid']
            timestamp = rev_doc['timestamp']
            user = rev_doc['user']
            userid = str(rev_doc['userid']) #to avoid user id overlaps across projects
            counter += 1
            if userid != '0':  
                editors[user] = editors.get(user,0) + 1
            else:
                ananoymous[user] = ananoymous.get(user,0) +1

    return editors,ananoymous

In [70]:
#there are some errors/warnings. Don't worry, we are getting a lower-bound
totalEditors = set()
anononymous = set()
startDate = '2020-01-01T00:00:00Z'
for project, pages in related_dict.items():
    print(project)
    for page in pages:
        try:
            result= countEditors(page,project,startDate)
            totalEditors = totalEditors.union(result[0].keys())
            anononymous = anononymous.union(result[1].keys())            
        except:
            pass


be_x_old.wikipedia
ennews.wikipedia
atj.wikipedia
tl.wikipedia
hak.wikipedia
ast.wikipedia
itvoyage.wikipedia
wuu.wikipedia
enquote.wikipedia
itquote.wikipedia
am.wikipedia
hy.wikipedia
el.wikipedia
mwl.wikipedia
oc.wikipedia
sh.wikipedia
esversity.wikipedia
pap.wikipedia
ko.wikipedia
zh.wikipedia
frquote.wikipedia
sat.wikipedia
frvoyage.wikipedia
ukquote.wikipedia
mn.wikipedia
or.wikipedia
frnews.wikipedia
ptnews.wikipedia
ceb.wikipedia
se.wikipedia
sd.wikipedia
konews.wikipedia
bcl.wikipedia
zh_classical.wikipedia
fi.wikipedia
qu.wikipedia
esvoyage.wikipedia
mg.wikipedia
gl.wikipedia
tr.wikipedia
war.wikipedia
ro.wikipedia
id.wikipedia
esnews.wikipedia
ta.wikipedia
rue.wikipedia
eu.wikipedia
mnw.wikipedia
finews.wikipedia
pa.wikipedia
azb.wikipedia
uknews.wikipedia
ne.wikipedia
data.wikipedia
wa.wikipedia
lij.wikipedia
et.wikipedia
zh_yue.wikipedia
ug.wikipedia
zhnews.wikipedia
sw.wikipedia
frp.wikipedia
ha.wikipedia
eml.wikipedia
km.wikipedia
enversity.wikipedia
species.wikipedia
a

In [71]:
print('Number registered editors %s' %len(totalEditors))
print('Number IPs  (anonymous) %s' %len(anononymous))

Number registered editors 16651
Number IPs  (anonymous) 18166


## Pageviews

In [10]:
%reload_ext rpy2.ipython

In [6]:
%%R
library(waxer)
library(data.table)
library(tidyverse)
library(lubridate)

/home/conniecc1/venv/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: data.table 1.12.6 using 16 threads (see ?getDTthreads).  Latest news: r-datatable.com

  warnings.warn(x, RRuntimeWarning)
/home/conniecc1/venv/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: ── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

  warnings.warn(x, RRuntimeWarning)
/home/conniecc1/venv/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: ✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0

  warnings.warn(x, RRuntimeWarning)
/home/conniecc1/venv/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()   masks data.table::between()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::

In [19]:
%%R -i related_pages

related_pages = data.table(related_pages)

pv_start = "20191201"
pv_end = "20200401"

pageviews = tibble(project = 'na', page_name = 'na', date = as.Date('2019-12-01'), views = 0)

for (row in 1:nrow(related_pages))
{

    project_name =  related_pages[row,"project"]
    page = related_pages[row,"page"]
    
    
      try({  
          
          result = wx_page_views(
            project = toString(project_name[[1]]),
            page_name = toString(page[[1]]),
            start_date = pv_start,
            end_date = pv_end )
           
       pageviews = bind_rows(pageviews,result)}, silent = TRUE)

} 
 

In [94]:
%%R 

#pv_project = pageviews %>% group_by(month=floor_date(date, "month"),value) %>%
#   summarize(total_views =sum(views)) 

#write.csv(pv_project,"pageviews by project.csv")

In [104]:
language = spark.run("""
    SELECT
      database_code, 
      domain_name as project,
      w.language_code, 
      w.language_name
    FROM canonical_data.wikis w """)

In [119]:
%%R -i language

language = as.tibble(language)
language = as.tibble(lapply (language,as.character))
language$project = substr(language$project, 1,nchar(language$project)-4)

In [122]:
%%R 
    
pv_language = merge(pv_project,language, by.x= "value", by.y = "project", all.x = TRUE)
pv_language  = pv_language %>% select(month, total_views, language_name) %>% group_by (month, language_name) %>%
   summarize(views =sum(total_views)) %>%
    spread(month, views)

write.csv(pv_language,"pageviews by language.csv")